In [1]:
# insert matplotlib in colab
%matplotlib inline

In [2]:
import torch
from torch import  nn
from torch.utils.data import  DataLoader
# als many domian specific lib,like torchaudio, torchtext
from torchvision import datasets
# datasets: CIFAR, COCO, etc
# also torchvision.datasets include 2 argument: transform, taget_transform to modify the samples
from torchvision.transforms import ToTensor

In [3]:
#Dowload training data from open datasets
#!datasets.FashionMNIST?
training_data = datasets.FashionMNIST(
   root = "data",
   train = True,
   download = True,
   transform = ToTensor()
)

test_data = datasets.FashionMNIST(
   root = "data",
   train = False,
   download = True,
   transform = ToTensor()
)

In [4]:
#pass the data to Dataloader
#Dataloader wraps an iterable over our dataset, and support automatic batching, sampling, shulffing and multiprocessing data loading
#!DataLoader?
#batch size is number of samples processed before model updated
#small batch size, need more epoched to convergent, large may miss the local lowest point.
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(dataset=training_data, batch_size=batch_size)
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size)

""" for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break """

' for X, y in test_dataloader:\n    print(f"Shape of X [N, C, H, W]: {X.shape}")\n    print(f"Shape of y: {y.shape} {y.dtype}")\n    break '

In [3]:
# Get cpu or gpu device for training.
# mps: meta performance shaders for macos with Apple silicon
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

# Define model
# Inherits from nn.Module
# nn.Module: base class for all nn module, your model should also subclass this class, nest them in a tree structure
class NeuralNetwork(nn.Module):
    #define the layers in NN
    def __init__(self):
        #super() with __init__()
        super().__init__()
        self.flatten = nn.Flatten()
        #logits
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    #define how the data pass the model
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
# loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(),  lr=1e-3)

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.291965  [    0/60000]
loss: 2.282793  [ 6400/60000]
loss: 2.272306  [12800/60000]
loss: 2.270894  [19200/60000]
loss: 2.237471  [25600/60000]
loss: 2.218630  [32000/60000]
loss: 2.224408  [38400/60000]
loss: 2.191149  [44800/60000]
loss: 2.181216  [51200/60000]
loss: 2.155266  [57600/60000]
Test Error: 
 Accuracy: 43.2%, Avg loss: 2.150735 

Epoch 2
-------------------------------
loss: 2.155892  [    0/60000]
loss: 2.150447  [ 6400/60000]
loss: 2.097304  [12800/60000]
loss: 2.113942  [19200/60000]
loss: 2.054028  [25600/60000]
loss: 2.001182  [32000/60000]
loss: 2.029241  [38400/60000]
loss: 1.949652  [44800/60000]
loss: 1.950255  [51200/60000]
loss: 1.883892  [57600/60000]
Test Error: 
 Accuracy: 55.3%, Avg loss: 1.882062 

Epoch 3
-------------------------------
loss: 1.913785  [    0/60000]
loss: 1.886065  [ 6400/60000]
loss: 1.771947  [12800/60000]
loss: 1.810238  [19200/60000]
loss: 1.692853  [25600/60000]
loss: 1.656658  [32000/600

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

Saved PyTorch Model State to model.pth


<All keys matched successfully>

In [11]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
